In [14]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import os
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
#import matplotlib.pyplot as plt
import numpy as np
import os.path
import csv
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
from sklearn import preprocessing
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from random import randint
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import math
import time
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import random
from metrics.Correlation import Correlation
from IO.Output import Output
from IO.Input import Input
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet
import DataSet.SnpsSelection as s

def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    return ids

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    print("ok1")
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    
    
    n = confusion_matrix.sum()
    print("n  = ",n)
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    
    print("val = ",phi2corr)
    
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


def cramers_stat(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()

    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))


def createCrammerTable(X):
    
    cram = np.zeros((len(X.T),len(X.T)), dtype = np.float32)
    
    y = np.zeros((2,len(X)), dtype = np.int32)
    
    for i in range(len(X.T)):
        print("i = ",i)
        for j in range(i,len(X.T)):
            
            y[0,:] = X[:,i]
            y[1,:] = X[:,j]
            
            v = cramers_stat(y)
            
            cram[i,j] = v
            cram[j,i] = v
            
    return cram


################################################################################################################################
################################################################################################################################
################################################################################################################################

def tables(sampleX,sampleY,k):
  
    samples = {}
    
    for run in range(1,k+1):
        
        d1 = {}
        

        dataTestX = sampleX[run]
        dataTestY = sampleY[run]

        n = 0

        for i in sampleX.keys():

            if i != run:

                n += len(sampleX[i])

        dataTrainX = np.zeros((n,len(sampleX[1].T)),dtype = int)
        dataTrainY = np.zeros((n,),dtype = int)

        count = 0

        for sample in sampleX.keys():

            if sample != run:

                 for i in range(len(sampleX[sample])):
                    for j in range(len(sampleX[sample].T)):
                        dataTrainX[count,j] = sampleX[sample][i,j]

                    dataTrainY[count] = sampleY[sample][i]
                    count += 1

        d1['trainX'] = dataTrainX
        d1['trainY'] = dataTrainY
        d1['testX'] = dataTestX
        d1['testY'] = dataTestY
        
        samples[run] = d1
    
    return samples
    
def kSampleData(k,X,Y):
    
    x = int (len(X) / k)
    allElements = np.zeros((len(X),),dtype = int)
    
    count1 = 1
    sampleX = {}
    sampleY = {}
   
    
    while count1 <= k:
        count2 = 1
        sampleData = []
        
        if count1 == k:
            x =  len(X) - ((k-1) * x)
        
        dataX = np.zeros((x,len(X.T)),dtype = int)
        dataY = np.zeros((x,),dtype = int)
        
        while count2 <= x:
            
            aRand = randint(0,len(X)-1)
            
            while allElements[aRand] == 1:
                
                aRand = randint(0,len(X)-1)
            
            allElements[aRand] = 1
            sampleData.append(aRand)
            count2 += 1
            
        for i in range(len(sampleData)):
            for j in range(len(X.T)):
                dataX[i,j] = X[sampleData[i],j]
            
            dataY[i] = Y[sampleData[i]]
            
        sampleX[count1] = dataX
        sampleY[count1] = dataY
        count1 +=1
        
    return tables(sampleX,sampleY,k)




def createNewTable(snps,X):
    
    newX = np.zeros((len(X),len(snps)),dtype = np.int32)
    count=0
    
    
    
    for i in range(len(newX)):
        for j in range(len(newX.T)):
            newX[i,j] = -1
    
    for i in range(len(snps)):
        
        newX[:,i] = X[:,snps[i]]
        
        
    print("new shape = ",newX.shape)
            
    return newX 



def featuresIds(oldSnps,snps):
    
    features = {}
    
    for i in range(len(snps)):
        features[i] = snps[i]
        
    return features



def crossValidiation(X, Y, k = 1, continious = True, classifier = None,OLS = False,Logistic = False):
    
    if not classifier:
        print("wrong!!!!!!! you have to choise a classifier")
        return
    
    results = {}
    accuracy = {}
    auc = {}
    recall = {}
    precision = {}
    f1Score = {}
    
    sumResults = 0.0
    sumAccuracy = 0.0
    sumAuc = 0.0
    sumRecall = 0.0
    sumPrecision = 0.0
    sumF1Score = 0.0
    
    samples = kSampleData(k,X,Y)
    
    for run in range(1, k + 1):
        
        trainX = samples[run]['trainX']
        trainY = samples[run]['trainY']
        
        #trainX,trainY = balancedData(trainX,trainY)
        
        testX = samples[run]['testX']
        testY = samples[run]['testY']
        
        
        if OLS:
            classifier = sm.OLS(trainY,trainX)
            yPredict = classifier.fit().predict(testX)
        else:

            classifier.fit(trainX, trainY)
            yPredict = classifier.predict(testX)
        
        if continious:
            
            for i in range(len(yPredict)):
                
                if (abs(0 - yPredict[i]) - abs(1 - yPredict[i])) <= 1e-10 :
                    yPredict[i] = 0
                else:
                    yPredict[i] = 1
                    
        if Logistic:
            
            probabilities = classifier.predict_proba(testX)
            
            for i in range(len(probabilities)):
                if probabilities[i][1] >= 0.8:
                    yPredict[i] = 1
                else:
                    yPredict[i] = 0
          
        accuracy[run] = metrics.accuracy_score(testY,yPredict)#(yPredict,testY)#
        fpr, tpr, thresholds = metrics.roc_curve(testY,yPredict)
        auc[run] = metrics.auc(fpr,tpr)
        recall[run] = metrics.recall_score(testY,yPredict)
        precision[run] = metrics.precision_score(testY,yPredict)
        f1Score[run] = f1_score(testY, yPredict, average='binary')
        
    
    for i in accuracy.keys():
        sumAccuracy = sumAccuracy + accuracy[i]
        sumAuc = sumAuc + auc[i]
        sumRecall = sumRecall + recall[i]
        sumPrecision = sumPrecision + precision[i]
        sumF1Score = sumF1Score + f1Score[i]
    
    results['accuracy'] = sumAccuracy / k
    results['auc'] = sumAuc / k
    results['recall'] = sumRecall / k
    results['precision'] = sumPrecision / k
    results['f1'] = sumF1Score / k
    
    return results


def writeCoef(path,snpsIds,sc,idToName,name = None):
        
        if not name:
            print("give a name to file")
            return
        
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt "  
    
        i=1
        while os.path.exists(p):
            
            p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
            i += 1
        
        snps = []
        for i in range(len(snpsIds)):
            s = snpsIds[i]
            snps.append(idToName[s])
            
        print("snpsIds = ",len(snpsIds))
        print("idToName = ",len(idToName))
        
        write = open(p,'w')
        for i in range(len(snps)):
            
            write.write(str(snps[i])+'\t'+str(sc[i])+'\n')
            
        write.close()
        
        
def writeNSemanticSnps(path, metric = None,columns = None, snps = None, name =None):
    
    if not name:
        print("give a name to file")
        return
        
    p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt"
    
    i=1
    while os.path.exists(p):
            
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
        i += 1
     
    write = open(p,'w')
    
    
    for i in range(len(snps)):
        s = snps[i]
        write.write(str(columns[s]) +',' + str(metric[i]) + '\n')
                
            
    write.close()


In [2]:
path = 'C:\\Users\\anton\\OneDrive\\Υπολογιστής\\'
df = open('C:\\Users\\anton\\OneDrive\\Υπολογιστής\\cram.txt','r')

columns = df.readline().split(',')[1:]
lines = df.readlines()

table = np.zeros((len(columns),len(columns)), dtype = np.float32)

for i in range(1,len(lines)):
    if i != 12002:
        row = lines[i].split(',')
        for j in range(1,len(row)):
            if i>12002:
                table[i-2,j-1] = float(row[j])
                
            else:
                table[i-1,j-1] = float(row[j])
        
        
df.close()
table

array([[1.    , 0.73  , 0.7344, ..., 0.738 , 0.7363, 0.735 ],
       [0.73  , 1.    , 0.7246, ..., 0.727 , 0.73  , 0.723 ],
       [0.7344, 0.7246, 1.    , ..., 0.727 , 0.728 , 0.7236],
       ...,
       [0.738 , 0.727 , 0.727 , ..., 1.    , 0.7266, 0.9106],
       [0.7363, 0.73  , 0.728 , ..., 0.7266, 1.    , 0.723 ],
       [0.735 , 0.723 , 0.7236, ..., 0.9106, 0.723 , 1.    ]],
      dtype=float32)

In [7]:
df = pd.DataFrame(table,columns = columns)

In [10]:
df.shape[0]

12002

In [13]:
thres = 0.8
down =99
up =100

snpsRed = []
snpsRed = s.lowCorrelation(table, threshold=thres, up=up, down=down)#oso megalytero threshold toso perissotero omoia einai 

count =  5517
len snpsRed =  5517


In [3]:
def _counterSnps(n):
    snpsCount = {}
    for i in range(n):
        snpsCount[i] = 0
    
    return snpsCount


def _highClass(X,b,c):

    snpsCount = _counterSnps(len(X.T))

    for i in range(len(X)):

        for j in range(i+1,len(X.T)):

            if X[i,j] - b < 1e-10 and X[i,j] >= c:

                snpsCount[i] = snpsCount[i] + 1
                snpsCount[j] = snpsCount[j] + 1

    return snpsCount


def highCorrelation(X, b,up,down, c=-2):

    snpsRed = []
    count = 0
    
    snpsCount = _highClass(X,b,c)

    
    return snpsCount

In [13]:
snpsRed = highCorrelation(table,0.7,100,100)

In [19]:
path = 'C:\\Users\\anton\\OneDrive\\Υπολογιστής\\'

ids={}
ids['coef']={}
    
coefs=list(snpsRed.values())
print("before = ",len(set(coefs)))
#for i in range(len(coefs)):
#   coefs[i] = abs(coefs[i])

print("after = ",len(set(coefs)))

idToName = {}
nameToId = {}

for i in range(len(coefs)):
    nameToId[coefs[i]] = []

for i in range(len(coefs)):
    nameToId[coefs[i]].append(i)
    idToName[i] = coefs[i]


ids['coef']['nameToId'] = nameToId
ids['coef']['idToName'] = idToName


sc = sorted(coefs,reverse=True)

top_30 = []

for i in range(30):

    snp = ids['coef']['nameToId'][sc[i]][0]
    ids['coef']['nameToId'][sc[i]].remove(snp)
    top_30.append(snpsRed[snp])
    

writeNSemanticSnps(path, metric = sc,columns = columns, snps = top_30, name ='nsemantic')

before =  313
after =  313
